### Ingest circuits.csv file


In [0]:
%run "../includes/configuration"

In [0]:
%run "../includes/common_functions"

##### Step 1 - Read the CSV file using the spark dataframe reader

In [0]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, DoubleType

In [0]:
circuits_schema = StructType(fields=[StructField("circuitId", IntegerType(), False),
                                     StructField("circuitRef", StringType(), True),
                                     StructField("name", StringType(), True),
                                     StructField("location", StringType(), True),
                                     StructField("country", StringType(), True),
                                     StructField("lat", DoubleType(), True),
                                     StructField("lng", DoubleType(), True),
                                     StructField("alt", IntegerType(), True),
                                     StructField("url", StringType(), True)
])

In [0]:
circuits_df = spark.read. \
              option("header",True). \
              schema(circuits_schema). \
              csv(f"{raw_folder_path}/circuits.csv")

In [0]:
display(circuits_df)

In [0]:
circuits_df.printSchema()

In [0]:
circuits_df.describe().show()

#### Step 1: Select only the required columns

In [0]:
circuits_selected_df = circuits_df.select("circuitId","circuitRef","name","location","country","lat","lng","alt")

In [0]:
from pyspark.sql.functions import col

circuits_selected_df = circuits_selected_df.select(col("circuitId"),col("circuitRef"),col("name"),col("location"),col("country"),col("lat"),col("lng"),col("alt"))

In [0]:
display(circuits_selected_df)

#### Step 2: Rename the columns as required

In [0]:
circuits_rename_df = circuits_selected_df.withColumnRenamed("circuitId", "circuit_id") \
    .withColumnRenamed("circuitRef", "circuit_ref") \
    .withColumnRenamed("name", "circuit_name") \
    .withColumnRenamed("lat", "latitude") \
    .withColumnRenamed("lng", "longitude") \
    .withColumnRenamed("alt", "altitude")


#### Step 3: Add ingestion data to the dataframe

In [0]:
circuits_final_df = add_ingestion_date(circuits_rename_df)

In [0]:
display(circuits_final_df)

#### Step 4: Write data to data lake as Parquet

In [0]:
circuits_final_df.write.mode("overwrite").parquet(f"{processed_folder_path}/circuits")

In [0]:
%fs
ls /mnt/formula1projectdl/processed/circuits

In [0]:
dbutils.notebook.exit("Success")